In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pytesseract as pt

pt.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update this path if necessary

In [3]:
import cv2 as cv
import numpy as np
import pytesseract as pt

def get_text_from_sign4(sign4):
    # Convert the image from BGR to RGB
    sign4_rgb = cv.cvtColor(sign4, cv.COLOR_BGR2RGB)
    sign4_gray = cv.cvtColor(sign4, cv.COLOR_BGR2GRAY)

    # Gaussian blur
    sign4_blur = cv.GaussianBlur(sign4_gray, (21, 21), 0)

    # Thresholding
    _, sign4_thresh = cv.threshold(sign4_blur, 47, 255, cv.THRESH_BINARY)

    # Edge detection
    sign4_edges = cv.Canny(sign4_thresh, 50, 150)

    # Create contours
    contours, hierarchy = cv.findContours(sign4_edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Sort by area, and take the top. Draw it in red.
    top_contour = sorted(contours, key=cv.contourArea, reverse=True)[0]

    # Let's convert the polygon represented by the biggest contour to a triangular polygon
    epsi = 0.02 * cv.arcLength(top_contour, True)
    polygon = cv.approxPolyDP(top_contour, epsi, True)

    # Adjust the polygon by bringing all triangle points in by 10
    adjustment_factor = 15

    # The top corner is the first sublist
    top = polygon[0][0]
    top_x = top[0]
    top_y = top[1]

    # The rightmost corner is the second sublist
    bottom = polygon[1][0]
    bottom_x = bottom[0]
    bottom_y = bottom[1]

    # The bottom corner is the third sublist
    right = polygon[2][0]
    right_x = right[0]
    right_y = right[1]

    # Now we need to take the top corner and increase the x by 10
    top_x += 10
    top_y += 10

    # The rightmost corner needs to decrease the x by 10 and increase the y by 10
    right_x -= 15
    right_y -= 15

    # The bottom corner needs to increase x by 10
    bottom_x += 10
    bottom_y -= 20

    # Update the polygon
    polygon[0][0] = [top_x, top_y]
    polygon[1][0] = [right_x, right_y]
    polygon[2][0] = [bottom_x, bottom_y]

    # First, we create a blank image.
    mask = np.zeros_like(sign4_gray)

    # Then, we draw the contours on the mask image
    cv.drawContours(mask, [polygon], -1, 255, thickness=cv.FILLED)

    # Apply the mask to the grayscale image to get the masked grey image
    masked_grey = cv.bitwise_and(sign4_gray, sign4_gray, mask=mask)

    # Invert the mask
    inverted_mask = cv.bitwise_not(mask)

    # Create a white background
    white_background = np.ones_like(sign4_gray) * 255

    # Combine the white background with the inverted mask
    outer_white = cv.bitwise_and(white_background, white_background, mask=inverted_mask)

    # Combine the masked grey image with the outer white image
    final_image = cv.bitwise_or(masked_grey, outer_white)

    # For final image, we threshold to just get black text on white background
    _, final_image_thresh = cv.threshold(final_image, 80, 255, cv.THRESH_BINARY)

    # Now finally use pytesseract to extract text
    text = pt.image_to_string(final_image_thresh)

    return text

# Example usage:
sign4 = cv.imread('sign4.jpg')
text = get_text_from_sign4(sign4)
print(text)

NQ-
PASSING
ZONE.

